In [1]:
import pandas as pd
import os
import numpy as np


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rc("font",family="Arial")

In [3]:
dftopo = pd.read_csv("all_subtype_all_region_r_values.csv",index_col=0)

In [4]:
for soma in ["soma_AP"]:
    for ter in ["ter_AP"]:
        dftmp = dftopo.loc[(dftopo.soma_label==soma)&(dftopo.ter_label==ter)]
        dfr_value = dftmp.pivot(index="subtype",columns="area",values="r_value").fillna(0)
        dfp_value = dftmp.pivot(index="subtype",columns="area",values="p_value").fillna(1)
        dfr_value.to_excel("%s_%s_r_values.xlsx"%(soma,ter))
        dfp_value.to_excel("%s_%s_p_values.xlsx"%(soma,ter))


        sorted_values = np.sort(dfp_value.values.flatten())
        sorted_values_new = sorted_values[sorted_values < 0.05]

        subtype_list = []
        area_list = []
        for i in sorted_values_new:
            position = dfp_value.isin([i])
            rows, cols = np.where(position)

            subtype = dfp_value.index[rows][0]
            area = dfp_value.columns[cols][0]

            if subtype not in subtype_list:
                subtype_list.append(subtype)
            if area not in area_list:
                area_list.append(area)

        dfp_value_new = dfp_value.loc[subtype_list,area_list]
        dfr_value_new = dfr_value.loc[subtype_list,area_list]
        
        fig, ax = plt.subplots(figsize = (18.5,9))
        sns.set(font_scale=1)
        sns.heatmap(data = dfr_value_new,cmap="bwr",vmin=-1,
                        vmax=1,linewidths=1, linecolor='black',annot=False,fmt="g",
                        mask = dfp_value_new>0.05
                        )
        # plt.savefig("heatmap_for_intra_three_gene.jpg",dpi =300)
        plt.savefig("%s_%s_for_All.pdf"%(soma,ter),dpi =600,format="pdf")
        plt.savefig("%s_%s_for_All.jpg"%(soma,ter),dpi =600,format="jpg")
        plt.close()
        
